In [1]:
from sklearn import datasets, model_selection, metrics
from scipy.special import expit
import pandas as pd
import numpy as np

# Функции активации и их производные

In [2]:
# функция sigmoid
def sigmoid(x):
    return expit(x)

In [3]:
# производная sigmoid
def sigmoid_derivative(x):
    return 1.0 * (1.0 - x)

In [4]:
# функция ReLu
def ReLu(x):
    return np.maximum(0, x)

In [5]:
# производная ReLu
def ReLu_derivative(x):
    return 1 * (x > 0)

# Многослойный персептрон

In [6]:
# Многослойный персептрон
class Perceptron:
    def __init__(self, n_inputs, n_outputs, n_layers, n_neurons, activation, derivative, max_iter=1000, eta=0.1):
        # кол-во входящих нейронов
        self.n_inputs = n_inputs
        # кол-во выходящих нейрноов
        self.n_outputs = n_outputs
        # число слоев
        self.n_layers = n_layers
        # число нейронов у слоя
        self.n_neurons = n_neurons
        # функция активации
        self.activation = activation
        # производная функции активации
        self.derivative = derivative
        # кол-во эпох обучения
        self.max_iter = max_iter
        # коэффициент обучения
        self.eta = eta
        # список слоев
        self.layers = []
        # инициализации слоев
        self.create_layers()
        
    # создание слоев    
    def create_layers(self):
        n_inputs = self.n_inputs
        for i in range(self.n_layers):
            self.layers.append(self.NeuronLayer(n_inputs, self.n_neurons))
            n_inputs = self.n_neurons
        self.layers.append(self.NeuronLayer(n_inputs, self.n_outputs))
        
    # прямое распространение
    def forward_propogation(self, inputs):
        for i in range(len(self.layers)):
            self.layers[i].values = self.activation(np.dot(np.transpose(self.layers[i].weights), inputs) 
                                                    + self.layers[i].bias)
            inputs = self.layers[i].values
        return inputs
    
    # обратное распространение
    def backward_propogation(self, inputs, error):
        self.layers[len(self.layers) - 1].error = np.multiply(error, 
                                                              self.derivative(self.layers[len(self.layers) - 1].values))
        
        for i in reversed(range(len(self.layers) - 1)):
            self.layers[i].error = np.multiply(np.dot(self.layers[i + 1].weights, self.layers[i + 1].error), 
                                               self.derivative(self.layers[i].values))
            
            
        for i in range(len(self.layers)):
            self.layers[i].weights_gradient = np.multiply(np.transpose(self.layers[i].error), inputs)
            self.layers[i].bias_gradient = self.layers[i].error
            inputs = self.layers[i].values
        
    # обучение сети
    def fit(self, X, y):
        for i in range(self.max_iter):
            weights_gradient_epoch = []
            bias_gradient_epoch = []
            n_iter = np.random.randint(1, X.shape[0])
            for j in range(len(self.layers)):
                weights_gradient_epoch.append(np.zeros((self.layers[j].weights.shape[0], self.layers[j].weights.shape[1])))
                bias_gradient_epoch.append(np.zeros((self.layers[j].bias.shape[0], self.layers[j].bias.shape[1])))
            for j in range(n_iter):
                index = np.random.randint(0, X.shape[0])
                inputs = X[index].reshape(-1, 1)
                outputs = self.forward_propogation(inputs)
                temp = np.array([0] * self.n_outputs).reshape(-1, 1)
                temp[y[index]] = 1
                error = outputs - temp
                self.backward_propogation(inputs, error)
                for k in range(len(self.layers)):
                    weights_gradient_epoch[k] += self.layers[k].weights_gradient
                    bias_gradient_epoch[k] += self.layers[k].bias_gradient
            for j in range(len(self.layers)):
                weights_gradient_epoch[j] /= n_iter
                self.layers[j].weights -= self.eta * weights_gradient_epoch[j]
                bias_gradient_epoch[j] /= n_iter
                self.layers[j].bias -= self.eta * bias_gradient_epoch[j]
    
    # предсказание значений
    def predict(self, X):
        outputs = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
            outputs[i] = self.forward_propogation(X[i].reshape(-1, 1)).argmax()
        return outputs
            
    # нейронный слой
    class NeuronLayer:
        def __init__(self, n_inputs, n_neurons):
            self.weights = np.random.normal(size=(n_inputs, n_neurons))
            self.bias = np.random.normal(size=(n_neurons, 1))
            self.weights_gradient = []
            self.bias_gradient = []
            self.values = []
            self.error = []

# Датасет iris

In [7]:
# получение датасета
iris = datasets.load_iris()
# создание датафрейма под датасет
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
# добавление целевого столбца
df_iris['target'] = pd.Series(iris.target)

In [8]:
# первые 5 записей из датафрейма
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [9]:
# получение целевых меток
y = df_iris['target'].values
# удаление целевого столбца из датафрейма
df_iris.pop('target')
# получение признаков
X = df_iris.values

In [10]:
# разделение на обучающуюся и тестовую выборки
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.20)

## Обучение моделей

### Multilayer perceptron

In [25]:
%%time
# Обучение модели
perceptron = Perceptron(X.shape[1], 3, 2, 2, sigmoid, sigmoid_derivative)
perceptron.fit(X_train, y_train)

Wall time: 4.62 s


###  K-Nearest Neighbors

In [12]:
%%time
from sklearn.neighbors import KNeighborsClassifier
# Обучение модели
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)

Wall time: 170 ms


KNeighborsClassifier(n_neighbors=3)

### Decision Tree Classifier

In [13]:
%%time
from sklearn import tree
# Обучение модели
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

Wall time: 34.8 ms


DecisionTreeClassifier()

### Naive Bayes Classifier

In [14]:
%%time
from sklearn.naive_bayes import GaussianNB
# Обучение модели
clf = GaussianNB()
clf.fit(X_train, y_train)

Wall time: 4 ms


GaussianNB()

### Support Vector Machines

In [15]:
%%time
from sklearn import svm
# Обучение модели
clf = svm.LinearSVC()
clf.fit(X_train, y_train)

Wall time: 13.5 ms


C:\Users\vano_\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC()

### Logistic Regression

In [16]:
%%time
from sklearn.linear_model import LogisticRegression
# Обучение модели
clf = LogisticRegression()
clf.fit(X_train, y_train)

Wall time: 72.8 ms


C:\Users\vano_\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

## Сравнение качества и времени работы моделей и MLP

In [17]:
from sklearn.metrics import classification_report, confusion_matrix

### Multilayer Perceptron и K-Nearest Neighbors

In [26]:
# предсказанные значения
y_pred = perceptron.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 0 10  0]
 [ 0 10  0]
 [ 0  5  5]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.40      1.00      0.57        10
           2       1.00      0.50      0.67        10

    accuracy                           0.50        30
   macro avg       0.47      0.50      0.41        30
weighted avg       0.47      0.50      0.41        30



In [27]:
# предсказанные значения
y_pred = clf.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



Время работы MLP - 4.62 s

Время работы K-Nearest Neighbors - 170 ms

K-Nearest Neighbors работает быстрее, чем MLP

Качество работы K-Nearest Neighbors лучше, чем у MLP

### Multilayer Perceptron и Decision Tree Classifier

In [28]:
# предсказанные значения
y_pred = perceptron.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 0 10  0]
 [ 0 10  0]
 [ 0  5  5]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.40      1.00      0.57        10
           2       1.00      0.50      0.67        10

    accuracy                           0.50        30
   macro avg       0.47      0.50      0.41        30
weighted avg       0.47      0.50      0.41        30



C:\Users\vano_\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
# предсказанные значения
y_pred = clf.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



Время работы MLP - 4.62 s

Время работы Decision Tree Classifier - 34.8 ms

Decision Tree Classifier работает быстрее, чем MLP

Качество работы Decision Tree Classifier лучше, чем у MLP

### Multilayer Perceptron и Naive Bayes Classifier

In [30]:
# предсказанные значения
y_pred = perceptron.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 0 10  0]
 [ 0 10  0]
 [ 0  5  5]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.40      1.00      0.57        10
           2       1.00      0.50      0.67        10

    accuracy                           0.50        30
   macro avg       0.47      0.50      0.41        30
weighted avg       0.47      0.50      0.41        30



C:\Users\vano_\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
# предсказанные значения
y_pred = clf.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



Время работы MLP - 4.62 s

Время работы Naive Bayes Classifier - 4 ms

Naive Bayes Classifier работает быстрее, чем MLP

Качество работы Naive Bayes Classifier лучше, чем у MLP

### Multilayer Perceptron и Support Vector Machines

In [32]:
# предсказанные значения
y_pred = perceptron.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 0 10  0]
 [ 0 10  0]
 [ 0  5  5]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.40      1.00      0.57        10
           2       1.00      0.50      0.67        10

    accuracy                           0.50        30
   macro avg       0.47      0.50      0.41        30
weighted avg       0.47      0.50      0.41        30



C:\Users\vano_\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
# предсказанные значения
y_pred = clf.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



Время работы MLP - 4.62 s

Время работы Support Vector Machines - 13.5 ms

Support Vector Machines работает быстрее, чем MLP

Качество работы Support Vector Machines лучше, чем у MLP

### Multilayer Perceptron и Logistic Regression

In [34]:
# предсказанные значения
y_pred = perceptron.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 0 10  0]
 [ 0 10  0]
 [ 0  5  5]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.40      1.00      0.57        10
           2       1.00      0.50      0.67        10

    accuracy                           0.50        30
   macro avg       0.47      0.50      0.41        30
weighted avg       0.47      0.50      0.41        30



C:\Users\vano_\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
# предсказанные значения
y_pred = clf.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.90      0.95        10
           2       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



Время работы MLP - 4.62 s

Время работы Logistic Regression - 72.8 ms

Logistic Regression работает быстрее, чем MLP

Качество работы Logistic Regression лучше, чем у MLP

# Датасет titanic

In [36]:
# получение датасета
df_titanic = pd.read_csv('../data/titanic.csv')

In [37]:
# первые 5 записей
df_titanic.head()

,PassengerID,Name,PClass,Age,Sex,Survived,SexCode
0,1,"Allen, Miss Elisabeth Walton",1st,29.00,female,1,1
1,2,"Allison, Miss Helen Loraine",1st,2.00,female,0,1
2,3,"Allison, Mr Hudson Joshua Creighton",1st,30.00,male,0,0
3,4,"Allison, Mrs Hudson JC (Bessie Waldo Daniels)",1st,25.00,female,0,1
4,5,"Allison, Master Hudson Trevor",1st,0.92,male,1,0


In [38]:
# кол-во пропущенных элементов
df_titanic.isnull().sum()

PassengerID      0
Name             0
PClass           0
Age            557
Sex              0
Survived         0
SexCode          0
dtype: int64

In [39]:
# выбрасывание пропущенных значений
df_titanic = df_titanic.dropna()

In [40]:
# кол-во пропущенных элементов
df_titanic.isnull().sum()

PassengerID    0
Name           0
PClass         0
Age            0
Sex            0
Survived       0
SexCode        0
dtype: int64

In [41]:
# кодирование категориальных признаков
mapping = {'1st': 1,
           '2nd': 2,
           '3rd': 3 }

df_titanic['PClass'] = df_titanic['PClass'].map(mapping)

In [42]:
# кодирование категориальных признаков
mapping = {'female': 0,
            'male': 1 }
df_titanic['Sex'] = df_titanic['Sex'].map(mapping)

In [43]:
# удаление из датафрейма столбца 'PassengerID'
df_titanic.pop('PassengerID')

0          1
1          2
2          3
3          4
4          5
        ... 
1308    1309
1309    1310
1310    1311
1311    1312
1312    1313
Name: PassengerID, Length: 756, dtype: int64

In [44]:
# удаление из датафрейма столбца 'Name'
df_titanic.pop('Name')

0                        Allen, Miss Elisabeth Walton
1                         Allison, Miss Helen Loraine
2                 Allison, Mr Hudson Joshua Creighton
3       Allison, Mrs Hudson JC (Bessie Waldo Daniels)
4                       Allison, Master Hudson Trevor
                            ...                      
1308                               Zakarian, Mr Artun
1309                           Zakarian, Mr Maprieder
1310                                 Zenni, Mr Philip
1311                                 Lievens, Mr Rene
1312                                   Zimmerman, Leo
Name: Name, Length: 756, dtype: object

In [45]:
# первые 5 записей
df_titanic.head()

,PClass,Age,Sex,Survived,SexCode
0,1,29.00,0,1,1
1,1,2.00,0,0,1
2,1,30.00,1,0,0
3,1,25.00,0,0,1
4,1,0.92,1,1,0


In [46]:
# матрица признаков и столбец меток
y = df_titanic['Survived'].values
df_titanic.pop('Survived')
X = df_titanic.values

In [47]:
# разделение выборки на обучающуюся и тестовую
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

## Обучение моделей

### Multilayer perceptron

In [48]:
%%time
# обучение модели
perceptron = Perceptron(X.shape[1], 2, 2, 2, sigmoid, sigmoid_derivative)
perceptron.fit(X_train, y_train)

Wall time: 21.9 s


###  K-Nearest Neighbors

In [49]:
%%time
from sklearn.neighbors import KNeighborsClassifier
# обучение модели
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)

Wall time: 7.51 ms


KNeighborsClassifier(n_neighbors=3)

### Decision Tree Classifier

In [50]:
%%time
from sklearn import tree
# обучение модели
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

Wall time: 2 ms


DecisionTreeClassifier()

### Naive Bayes Classifier

In [51]:
%%time
from sklearn.naive_bayes import GaussianNB
# обучение модели
clf = GaussianNB()
clf.fit(X_train, y_train)

Wall time: 7.07 ms


GaussianNB()

### Support Vector Machines

In [52]:
%%time
from sklearn import svm
# обучение модели
clf = svm.LinearSVC()
clf.fit(X_train, y_train)

Wall time: 16 ms


C:\Users\vano_\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC()

### Logistic Regression

In [53]:
%%time
from sklearn.linear_model import LogisticRegression
# обучение модели
clf = LogisticRegression()
clf.fit(X_train, y_train)

Wall time: 18 ms


LogisticRegression()

## Сравнение качества и времени работы моделей и MLP

In [54]:
from sklearn.metrics import classification_report, confusion_matrix

### Multilayer Perceptron и K-Nearest Neighbors

In [55]:
# предсказанные значения
y_pred = perceptron.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[88  2]
 [57  5]]
              precision    recall  f1-score   support

           0       0.61      0.98      0.75        90
           1       0.71      0.08      0.14        62

    accuracy                           0.61       152
   macro avg       0.66      0.53      0.45       152
weighted avg       0.65      0.61      0.50       152



In [56]:
# предсказанные значения
y_pred = clf.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[72 18]
 [22 40]]
              precision    recall  f1-score   support

           0       0.77      0.80      0.78        90
           1       0.69      0.65      0.67        62

    accuracy                           0.74       152
   macro avg       0.73      0.72      0.72       152
weighted avg       0.73      0.74      0.74       152



Время работы MLP - 21.9 s

Время работы K-Nearest Neighbors - 7.51 ms

K-Nearest Neighbors работает быстрее, чем MLP

Качество работы K-Nearest Neighbors лучше, чем у MLP

### Multilayer Perceptron и Decision Tree Classifier

In [57]:
# предсказанные значения
y_pred = perceptron.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[88  2]
 [57  5]]
              precision    recall  f1-score   support

           0       0.61      0.98      0.75        90
           1       0.71      0.08      0.14        62

    accuracy                           0.61       152
   macro avg       0.66      0.53      0.45       152
weighted avg       0.65      0.61      0.50       152



In [58]:
# предсказанные значения
y_pred = clf.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[72 18]
 [22 40]]
              precision    recall  f1-score   support

           0       0.77      0.80      0.78        90
           1       0.69      0.65      0.67        62

    accuracy                           0.74       152
   macro avg       0.73      0.72      0.72       152
weighted avg       0.73      0.74      0.74       152



Время работы MLP - 21.9 s

Время работы Decision Tree Classifier - 2 ms

Decision Tree Classifier работает быстрее, чем MLP

Качество работы Decision Tree Classifier лучше, чем у MLP

### Multilayer Perceptron и Naive Bayes Classifier

In [59]:
# предсказанные значения
y_pred = perceptron.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[88  2]
 [57  5]]
              precision    recall  f1-score   support

           0       0.61      0.98      0.75        90
           1       0.71      0.08      0.14        62

    accuracy                           0.61       152
   macro avg       0.66      0.53      0.45       152
weighted avg       0.65      0.61      0.50       152



In [60]:
# предсказанные значения
y_pred = clf.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[72 18]
 [22 40]]
              precision    recall  f1-score   support

           0       0.77      0.80      0.78        90
           1       0.69      0.65      0.67        62

    accuracy                           0.74       152
   macro avg       0.73      0.72      0.72       152
weighted avg       0.73      0.74      0.74       152



Время работы MLP - 21.9 s

Время работы Naive Bayes Classifier - 7.07 ms

Naive Bayes Classifier работает быстрее, чем MLP

Качество работы Naive Bayes Classifier лучше, чем у MLP

### Multilayer Perceptron и Support Vector Machines

In [61]:
# предсказанные значения
y_pred = perceptron.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[88  2]
 [57  5]]
              precision    recall  f1-score   support

           0       0.61      0.98      0.75        90
           1       0.71      0.08      0.14        62

    accuracy                           0.61       152
   macro avg       0.66      0.53      0.45       152
weighted avg       0.65      0.61      0.50       152



In [62]:
# предсказанные значения
y_pred = clf.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[72 18]
 [22 40]]
              precision    recall  f1-score   support

           0       0.77      0.80      0.78        90
           1       0.69      0.65      0.67        62

    accuracy                           0.74       152
   macro avg       0.73      0.72      0.72       152
weighted avg       0.73      0.74      0.74       152



Время работы MLP - 21.9 s

Время работы Support Vector Machines - 16 ms

Support Vector Machines работает быстрее, чем MLP

Качество работы Support Vector Machines лучше, чем у MLP

### Multilayer Perceptron и Logistic Regression

In [64]:
# предсказанные значения
y_pred = perceptron.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[88  2]
 [57  5]]
              precision    recall  f1-score   support

           0       0.61      0.98      0.75        90
           1       0.71      0.08      0.14        62

    accuracy                           0.61       152
   macro avg       0.66      0.53      0.45       152
weighted avg       0.65      0.61      0.50       152



In [65]:
# предсказанные значения
y_pred = clf.predict(X_test)
# матрицы характеристик
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[72 18]
 [22 40]]
              precision    recall  f1-score   support

           0       0.77      0.80      0.78        90
           1       0.69      0.65      0.67        62

    accuracy                           0.74       152
   macro avg       0.73      0.72      0.72       152
weighted avg       0.73      0.74      0.74       152



Время работы MLP - 21.9 s

Время работы Logistic Regression - 18 ms

Logistic Regression работает быстрее, чем MLP

Качество работы Logistic Regression лучше, чем у MLP